# Common rules
1. DO NOT change the variables names which are formatted as `df_xx`
2. You can display the data using the `.show()` function

# Main contents

##### Question 1: Read CSV File
1. Read the CSV file into a DataFrame namely `df`
2. Print the schema of DataFrame `df`.
3. Cache this `df` into memory for reuses below.
4. Get the first 5 rows of the DataFrame `df` to driver

In [1]:
from pyspark.sql import SparkSession
# the following Spark session creation contains jars which support writing to Delta format
# note: checkout this link to see which version of Delta is compatible with your version of Pyspark: https://docs.delta.io/latest/releases.html
spark = SparkSession.builder	\
	.config('spark.jars.packages', 'io.delta:delta-spark_2.12:3.2.0')	\
	.config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')	\
	.config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')	\
	.getOrCreate()

In [ ]:
df = spark.read.format('csv').option('header', 'true').load('exams/data/sample.csv')

df.printSchema()

df = df.cache()

df.head(5)

##### Question 2: Count Rows
Count the number of rows in the DataFrame.

In [ ]:
row_count = df.count()
print("Number of rows:", row_count)

##### Question 3: Filter Rows
Filter rows where the column 'age' is greater than 30.

In [ ]:
df_03 = df.filter('age > 30')
df_03.show()

##### Question 4: Select Columns
Select only the columns 'name' and 'age'.

In [ ]:
df_04 = df.select('name', 'age')
df_04.show()

##### Question 5: Group By and Aggregate
Group by 'department' and calculate the average 'salary'.

In [ ]:
from pyspark.sql.functions import avg
df_05 = df.groupBy('department').agg(avg('salary'))
df_05.show()

##### Question 6: Rename Column
Rename the column 'name' to 'new_name'.

In [ ]:
df_06 = df.withColumnRenamed('name', 'new_name')
df_06.show()

##### Question 7: Drop Column
Drop the column 'salary' from the DataFrame.

In [ ]:
df_07 = df.drop('salary')
df_07.show()

##### Question 8: Add New Column
Add a new column 'bonus' which is 10% of 'salary'.

In [ ]:
from pyspark.sql.functions import col
df_08 = df.withColumn('bonus', 0.1 * col('salary'))

df_08.show()

##### Question 9: Sort DataFrame
Sort the DataFrame by 'age' in descending order.

In [ ]:
from pyspark.sql.functions import desc
df_09 = df.orderBy(desc('age'))
df_09.show()

##### Question 10: Write DataFrame to Parquet
Write the DataFrame to a Parquet file named "output/data.parquet"

In [ ]:
df.write.parquet('exams/12345678/output/data.parquet')

##### Question 11: Read JSON File
Read a JSON file into a DataFrame.

In [ ]:
df_11 = spark.read.option('multiLine', 'true').json('exams/data/sample.json')
df_11.show()

##### Question 12: Drop Duplicates
Drop duplicate rows based on the 'name' column.

In [ ]:
df_12 = df.dropDuplicates(['name'])
df_12.show()

##### Question 13: Check for Null Values
1. Check for null values in the 'email' column.
2. Drop the null values

In [ ]:
df_13_1 = df.filter(col('email').isNull())
df_13_1.show()

df_13_2 = df.dropna(subset = ['email'])
df_13_2.show()

##### Question 14: Create Temporary View
Create a temporary view named 'people_view' from the DataFrame.

In [ ]:
df.createOrReplaceTempView('people_view')
df_14 = spark.sql("SELECT * FROM people_view")
df_14.show()

##### Question 15: SQL Query on DataFrame
Execute an SQL query to select names of people older than 25.

In [ ]:
df_15 = spark.sql('select * from people_view where age > 25')
df_15.show()

##### Question 16: Join Two DataFrames using Spark SQL
1. Loading data from `exams/data/department.csv` into a temporary view named `department`
2. Join two DataFrames on the 'id' column to find names of department managers of each department. Choose the join type which eliminate the null values

In [ ]:
tmp_df = spark.read.option('header', 'true').csv('exams/data/department.csv').cache()

tmp_df.createOrReplaceTempView('department')
df_16 = spark.sql('select d.*, p.name from department d inner join people_view p on d.manager_id = p.id')
df_16.show()


##### Question 17: Save DataFrame as Delta Table
Save the DataFrame as a Delta table.

In [2]:
df.write.format("delta").save("exams/output/delta_table")

##### Question 18: Read Delta Table
Read a Delta table into a DataFrame.

In [ ]:
df_18 = spark.read.format('delta').load('exams/output/delta_table')
df_18.show()

##### Question 19: Use UDF (User Defined Function)
Define a UDF to convert names to uppercase and apply it to the `name` column. Name the udf-applied column as `upper_name`.

In [ ]:
from pyspark.sql.functions import udf

upper_udf = udf(lambda x: x.upper())
df_19 = df.select(upper_udf(df.name).alias('upper_name'))
df_19.show()

##### Question 20: Extract Year from Date Column
Extract the year from a date column 'hire_date' as a new column named 'hire_year'

In [ ]:
from pyspark.sql.functions import year

df_20 = df.select(year(df.hire_date).alias("hire_year"))
df_20.show()

##### Question 21: Create a New DataFrame from Existing Columns
Create a new DataFrame with only 'name', 'age', and 'salary'.

In [ ]:
df_21 = df.select('name', 'age', 'salary')
df_21.show()

##### Question 22: Pivot Table
Create a pivot table to show total salary by department.

In [ ]:
from pyspark.sql.functions import col
# first, cast the salary column to double type
df = df.withColumn('salary', col('salary').cast('int'))
df_22 = df.groupBy("department").pivot("gender").sum('salary').alias('total_salary')
df_22.drop('null').show()

##### Question 23: Handle Missing Values
Fill missing values in the 'salary' column with the average salary.

In [ ]:
from pyspark.sql.functions import avg
avg_salary = df.select(avg('salary')).collect()[0][0]
print(avg_salary)
df_23 = df.fillna(value=avg_salary, subset=['salary'])
df_23.show()

##### Question 24: Convert String to Date Type
Convert the string column 'hire_date' to DateType.

In [ ]:
from pyspark.sql.functions import to_date
df_24 = df.withColumn('hire_date', to_date('hire_date', 'yyyy-MM-dd'))
df_24.printSchema()
df_24.show()

##### Question 25: Count Distinct Values
Count distinct values in the 'department' column.

In [ ]:
distinct_count = df.select('department').distinct().count()
print(distinct_count)

##### Question 26: Find Maximum Value in a Column
Find the maximum value in the 'salary' column.

In [ ]:
from pyspark.sql.functions import max
df_26 = df.select(max('salary'))
df_26.show()

##### Question 27: Use Window Functions


In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

window_spec = Window.partitionBy("department").orderBy(df.salary.desc())
df_27 = df.withColumn("rank", rank().over(window_spec))
df_27.show()

##### Question 28: Save DataFrame as CSV with Options
Save the DataFrame as a CSV file with header and compression options.

In [35]:
df.write.option("header", "true").option("compression", "gzip").csv("exams/output/compressed_data.csv")

##### Question 29: Read Multiple Files into a Single DataFrame
Read multiple CSV files from a directory into a single DataFrame.

In [ ]:
df_29 = spark.read.csv("exams/data/*.csv", header=True)
df_29.show()